In [2]:
import requests
from urllib.parse import urlparse #한글처리
import urllib.parse
from tqdm import tqdm
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import logging
import traceback
from datetime import datetime
import json
from hdfs import InsecureClient
client = InsecureClient('http://localhost:9870', user='big')

In [ ]:
df = pd.read_csv('파일이름', encoding='utf-8')

In [ ]:
df

In [3]:
url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=201641'

In [61]:
# 관람객 평점
aud_score=soup.findAll("span",{"class":"st_on"})[0].text.split(" ")[2].replace('점',"")
aud_score

'8.03'

In [50]:
# 전문가 평점
exp_score=soup.findAll("div",{"class":"spc_score_area"})[0].text.split("\n\n")[2]
exp_score

'5.50'

In [59]:
# 네티즌 평점
# star_score 
ntz_score=soup.findAll("a",{"id":"pointNetizenPersentBasic"})[0].text
ntz_score

'8.62'

## HDFS 저장

In [5]:
date=str(datetime.now().date())
file_dir = '/movie_data/score/'

cols = ['AUDI_SC', 'EXPE_SC', 'NETI_SC', 'STD_DATE']


for i in tqdm(range(0,1)):
    try :
        data=[]
        file_name = 'movie_score_' + date + '.json'
        
        # 페이지 요청
        html = requests.get(url).content
        # 파싱객체 생성
        soup = BeautifulSoup(html,"html.parser")
        time.sleep(1)
        
        rows=[]
        
        aud=soup.findAll("span",{"class":"st_on"})[0].text.split(" ")[2].replace('점',"")
        spc=soup.findAll("div",{"class":"spc_score_area"})[0].text.split("\n\n")[2]
        ntz=soup.findAll("a",{"id":"pointNetizenPersentBasic"})[0].text
        
        rows.append(aud)
        rows.append(spc)
        rows.append(ntz)
        rows.append(date)
        
        tmp = dict(zip(cols, rows))
        data.append(tmp)
        
        res = {
            'meta':{
                'desc':'네이버 영화 평점 현황',
                'cols':{
                    'AUDI_SC':'관람객평점'
                    ,'EXPE_SC':'기자및평론가평점'
                    ,'NETI_SC':'네티즌평점'
                    ,'STD_DATE':'수집일자'
                },
            },
           'data':data
        }
            
        client.write(file_dir+file_name, json.dumps(res, ensure_ascii=False), encoding='utf-8')
        
    except Exception as e:
        rows.append('-')


100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]


In [ ]:
data